# **Project Name**    -Cardiovascular Risk Prediction

##### **Project Type**    - Classification
##### **Contribution**    - Team
##### **Team Member 1 -** Ayushi Sharma
##### **Team Member 2 -** Palak

# **Project Summary -**

The Framingham dataset is a well-known dataset derived from an ongoing study of residents in Framingham, Massachusetts, aimed at predicting the risk of Coronary Heart Disease (CHD) within a 10-year period. This project focuses on building a classification model to predict CHD risk based on an individual’s medical and behavioral features. A key part of the project involves identifying optimal preprocessing techniques to prepare the dataset for effective machine learning analysis.

The dataset includes a variety of demographic data (such as sex, age, and education), behavioral data (like smoking habits), and medical information (including blood pressure, diabetes, cholesterol, BMI, glucose levels, and history of stroke or hypertension), all of which are relevant predictors of cardiovascular risk.

The initial step in the project involves comprehensive data preprocessing. This includes handling missing values, detecting and treating outliers, and performing feature engineering. Missing values are imputed using realistic, research-backed methods, while outliers are analyzed to determine if they fall within a plausible range before being addressed to minimize data loss. New features are engineered by combining existing variables, such as calculating mean arterial pressure and categorizing glucose levels. Hypothesis testing is used to evaluate the statistical significance and dependency of each feature in relation to the CHD outcome.

Following preprocessing, the dataset is divided into training and testing sets using a suitable test ratio. The features in both sets are scaled using MinMaxScaler, based on the distribution in the training set. Due to the imbalance in the target variable (CHD occurrence), SMOTE (Synthetic Minority Over-sampling Technique) is applied to the training set to enhance model training. Three classification algorithms are then trained: Logistic Regression, Decision Tree, and XGBoost. Each model’s performance is evaluated using the Recall score, chosen as the primary metric given the importance of identifying positive CHD cases. The model with the highest Recall on the test set is selected as the final model.

The Decision Tree model trained on the original dataset achieves the highest Test Recall score of 87.25%, outperforming all other models and preprocessing variations. This model is further interpreted using SHAP (SHapley Additive exPlanations) values, which reveal that age is the most influential predictor of CHD risk, while sex and diabetes contribute the least.

In summary, this project emphasizes the critical role of data preprocessing, feature engineering, and model selection in building an effective CHD risk prediction model. The use of SHAP values enhances model interpretability by identifying the most impactful features. Future work could involve incorporating additional features, experimenting with alternative preprocessing methods, and validating the model on external datasets.



# **GitHub Link -**

# **Problem Statement**


Cardiovascular diseases, particularly coronary heart disease (CHD), continue to be a major health concern worldwide. Early identification and risk stratification of individuals susceptible to developing CHD over a 10-year period is critical for targeted intervention and prevention strategies.

This project aims to build a predictive machine learning model using the Framingham Heart Study dataset to classify individuals into high-risk and low-risk categories for CHD. The challenge lies in dealing with imbalanced classes, limited positive cases, and ensuring clinical interpretability of the predictions. The final goal is to support healthcare decision-making by identifying high-risk individuals early, ultimately reducing the incidence and healthcare costs associated with coronary heart disease.

# **General Guidelines** : -  

1.   Well-structured, formatted, and commented code is required.
2.   Exception Handling, Production Grade Code & Deployment Ready Code will be a plus. Those students will be awarded some additional credits.
     
     The additional credits will have advantages over other students during Star Student selection.
       
             [ Note: - Deployment Ready Code is defined as, the whole .ipynb notebook should be executable in one go
                       without a single error logged. ]

3.   Each and every logic should have proper comments.
4. You may add as many number of charts you want. Make Sure for each and every chart the following format should be answered.
        

```
# Chart visualization code
```
            

*   Why did you pick the specific chart?
*   What is/are the insight(s) found from the chart?
* Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

5. You have to create at least 15 logical & meaningful charts having important insights.


[ Hints : - Do the Vizualization in  a structured way while following "UBM" Rule.

U - Univariate Analysis,

B - Bivariate Analysis (Numerical - Categorical, Numerical - Numerical, Categorical - Categorical)

M - Multivariate Analysis
 ]





6. You may add more ml algorithms for model creation. Make sure for each and every algorithm, the following format should be answered.


*   Explain the ML Model used and it's performance using Evaluation metric Score Chart.


*   Cross- Validation & Hyperparameter Tuning

*   Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

*   Explain each evaluation metric's indication towards business and the business impact pf the ML model used.




















# ***Let's Begin !***

##  ***1. Know Your Data***

### Import Libraries

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip install graphviz
import graphviz
from IPython.display import SVG, display
from graphviz import Source


from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import ttest_1samp, shapiro
from sklearn.feature_selection import chi2

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.metrics import recall_score, make_scorer, roc_auc_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb

!pip install shap
import shap

import warnings
warnings.filterwarnings('ignore')

### Dataset Loading

In [ ]:
# Load Dataset
df = pd.read_csv('data_cardiovascular_risk.csv')

### Dataset First View

In [ ]:
# Dataset First Look
df.head()

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
print(f'Number of rows in the dataset: {df.shape[0]}')
print(f'Number of columns  in the dataset: {df.shape[1]}')

### Dataset Information

In [ ]:
# Dataset Info
df.info()

#### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
print(f'Number of duplicated rows in the dataset: {df.duplicated().sum()}')

It is expected that **id** column should have all unique values, as it represents one unique person of interest

In [ ]:
df['id'].duplicated().sum()

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count
print(f'There are {df.isna().sum().sum()} missing values in the dataset\n')
df.isna().sum()

In [ ]:
# Visualizing the missing values
plt.figure(figsize=(13, 10))
sns.heatmap(df.isna())





### What did you know about your dataset?

On a first look at the dataset, it is found that
*   There are **3390 rows and 17 columns**, out of which one is **TenYearCHD** which is the the dependent variable to be predicted
*   Two of these features are **not in numerical (int/float) datatype**
*   There are **no duplicated data** in the dataset, and all values in **id** column are **unique**
*   There are **510 missing values** in the dataset, with **304** of them in **glucose** column

##  ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns
df.columns

In [ ]:
# Dataset Describe
df.describe()

### Variable Description

The various features (and their type), which are potential factors for CHD risk, utilised in this Cardiovascular Risk assessment are:

*   **id**: Personal identification number (Unique)

Demographic:
*   **sex**: Male or Female (Nominal)
*   **age**: Age of the patient (Continuous)
*   **education**: no information provided (**Ordinal assumed**)

Behavioral:
*   **is_smoking**: Whether or not the patient is a current smoker (Nominal)
*   **cigsPerDay**: Number of cigarettes smoked by the person per day on average (Continuous)

Medical information:
*   **BPMeds**: Whether or not the patient is on blood pressure medication (Nominal)
*   **prevalentStroke**: Whether or not the patient previously had a stroke (Nominal)
*   **prevalentHyp**: Whether or not the patient was hypertensive (Nominal)
*   **diabetes**: Whether or not the patient has diabetes (Nominal)
*   **totChol**: Total cholesterol level in mg/dL (Continuous)
*   **sysBP**: systolic blood pressure in mmHg (Continuous)
*   **diaBP**: diastolic blood pressure in mmHg (Continuous)
*   **BMI**: Body Mass Index (Continuous)
*   **heartRate**: Heart rate (Continuous)
*   **glucose**: glucose level in mg/dL (Continuous).

Target variable to predict:
*   **TenYearCHD**: 10 year risk of coronary heart disease (CHD) - (Nominal)



| Variable         | Description                                                  |
|------------------|--------------------------------------------------------------|
| `age`            | Age of the patient                                           |
| `education`      | Education level (1-4 scale)                                  |
| `sex`            | Gender (`M` or `F`)                                          |
| `is_smoking`     | Current smoking status (`YES` or `NO`)                       |
| `cigsPerDay`     | Number of cigarettes smoked per day                          |
| `BPMeds`         | Binary indicator for blood pressure medication use           |
| `prevalentStroke` | History of stroke                                            |
| `prevalentHyp`   | History of hypertension                                      |
| `diabetes`       | Diabetic status (1 = diabetic, 0 = not)                      |
| `totChol`        | Total cholesterol (mg/dL)                                    |
| `sysBP`          | Systolic blood pressure                                      |
| `diaBP`          | Diastolic blood pressure                                     |
| `BMI`            | Body Mass Index                                              |
| `heartRate`      | Heart rate (beats per minute)                                |
| `glucose`        | Glucose level (mg/dL)                                        |
| `TenYearCHD`     | 10-year risk of CHD (1 = yes, 0 = no)                        |

### Check Unique Values for each variable.

In [ ]:
# Splitting the categorical and continuous variables
categ_vars = ['sex', 'education', 'is_smoking', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes']
cont_vars = ['age', 'cigsPerDay', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']

In [ ]:
# Check Unique Values for categorical variables
for var in categ_vars:
  print(f'Unique values in {var} are: {df[var].dropna().unique()})')

In [ ]:
# Checking the values for id
print(f"The number of unique IDs in dataset are {df['id'].nunique()}, with the minimum as {df['id'].min()} and maximum as {df['id'].max()}")

It is observed that,
*   **Sex** contains two values - Male and Female
*   **Education** has 4 values - 1, 2, 3, 4. It is assumed that it represents a hierarchy of educational qualification levels among the patients
*   **is_smoking**, **BPMeds**, **prevalentStroke**, **prevalentHyp** and **diabetes** are nominal binaries where 0 represents 'No' and 1 represents 'Yes'
*   Each patient in the dataset are **unique** and there are no repeated data

## 3. ***Data Wrangling***

In [ ]:
#duplicate
df.drop_duplicates(inplace=True)    # No duplicates were present

### What all manipulations have you done and insights you found?

Data wrangling was not performed in this project in this section now as the objective was to explore and analyze the raw data in its original form for now. This approach allowed for an authentic understanding of the dataset's inherent structure and characteristics, which was essential for meeting the goals.

## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

In [ ]:
sns.set(style='darkgrid')
plt.rcParams['figure.figsize'] = (10, 6)

#### Chart - 1

In [ ]:
# Chart - 1 visualization code
sns.histplot(df['age'],kde=True,color='skyblue')
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

##### 1. Why did you pick the specific chart?

To understand patient age spread .

##### 2. What is/are the insight(s) found from the chart?

Most patients are between 40 to 60 years old, indicating a middle-aged demographic.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Yes.Helps target middle-aged adults for preventive care,
Yes.Older population indicates rising CHD risk and healthcare costs.

#### Chart - 2

In [ ]:
# Chart - 2 visualization code
sns.countplot(x='sex', data=df,palette='bright')
plt.title('Male vs Female Distribution')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.show()

##### 1. Why did you pick the specific chart?

Gender ratio influences disease prevelance.

##### 2. What is/are the insight(s) found from the chart?

Slightly more females than males are present in the dataset.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Yes.Supports gender-based health planning and resource allocation,
Yes.Female dominance with slightly higher CHD risk could impact healthcare resource distribution.

#### Chart - 3

In [ ]:
# Chart - 3 visualization code
if 'education' in df.columns:
    sns.countplot(x='education', data=df, palette='bright')
    plt.title('Education Level Distribution')
    plt.xlabel('Education Level')
    plt.ylabel('Count')
    plt.show()

##### 1. Why did you pick the specific chart?

Socio-economic status links to health awareness.

##### 2. What is/are the insight(s) found from the chart?

Majority of participants fall under lower education levels (levels 1 and 2).

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Yes.Enables health education campaigns based on literacy levels,
Yes.Low education levels correlate with poor health awareness and lifestyle habits.

#### Chart - 4

In [ ]:
# Chart - 4 visualization code
print(df.is_smoking.value_counts())
sns.countplot(x='is_smoking', data=df, palette='pastel')
plt.title('Smoking Status Distribution')
plt.xlabel('Smoking Status')
plt.ylabel('Count')
plt.show()

##### 1. Why did you pick the specific chart?

Smoking is a key cardiovascular risk.

##### 2. What is/are the insight(s) found from the chart?

There is a significant number of active smokers in the dataset.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Yes. Aids in designing anti-smoking health programs,
Yes. High smoking rate signals increased long-term health issues.

#### Chart - 5

In [ ]:
# Chart - 5 visualization code
sns.histplot(df['sysBP'],kde=True,color='orange')
plt.title('Systolic Blood Pressure Distribution')
plt.xlabel('Systolic BP')
plt.ylabel('Density')
plt.show()

##### 1. Why did you pick the specific chart?

BP is directly tied to heart disease.

##### 2. What is/are the insight(s) found from the chart?

Many individuals show high-normal or elevated systolic blood presuure.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Yes. Promotes early intervention for blood pressure management,
Yes. High prevalence of elevated BP indicates future chronic illness.

#### Chart - 6

In [ ]:
# Chart - 6 visualization code
sns.boxplot(x='TenYearCHD',y='age',data=df,palette='coolwarm')
plt.title('Age vs TenYearCHD')
plt.xticks([0,1],['No CHD','CHD'])
plt.ylabel('Age')
plt.show()

##### 1. Why did you pick the specific chart?

To find out if age affects CHD risk.

##### 2. What is/are the insight(s) found from the chart?

CHD-positive patients tend to be older than CHD-negative ones.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Yes. Justifies focused screening for older adults,
Yes. Older CHD patients may reduce workforce productivity.

#### Chart - 7

In [ ]:
# Chart - 7 visualization code
sns.countplot(x='sex',hue='TenYearCHD',data=df, palette='Set1')
plt.xticks([0,1],['Female','Male'])
plt.title('Gender vs CHD Risk')
plt.ylabel('Count')
plt.show()

##### 1. Why did you pick the specific chart?

Gender may correlate with heart disease.

##### 2. What is/are the insight(s) found from the chart?

Males show a slightly higher incidence of CHD compared to females.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Yes. Encourages gender-specific heart health strategies,
Yes. Males show higher CHD risk, possibly impacting male labor force.


#### Chart - 8

In [ ]:
# Chart - 8 visualization code
sns.boxplot(x='TenYearCHD',y='totChol',data=df,palette='Accent')
plt.title('Total Cholesterol vs CHD')
plt.xticks([0,1],['No CHD','CHD'])
plt.ylabel('Total Cholesterol')
plt.show()

##### 1. Why did you pick the specific chart?

Cholesterol is a classic risk factor.

##### 2. What is/are the insight(s) found from the chart?

CHD patients have higher total cholesterol levels on average.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Yes. Validates the need for regular lipid screening,
Yes. High cholesterol in CHD patients suggests long-term care dependency.

#### Chart - 9

In [ ]:
# Chart - 9 visualization code
sns.boxplot(x='TenYearCHD',y='glucose',data=df,palette='cubehelix')
plt.title('Glucose Levels vs CHD')
plt.xticks([0,1],['No CHD','CHD'])
plt.ylabel('Glucose')
plt.show()

##### 1. Why did you pick the specific chart?

Diabetes and CHD are related.

##### 2. What is/are the insight(s) found from the chart?

Glucose levels are slightly higher in CHD-positive individuals.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Yes. Supports dual screening for diabetes and heart disease,
Yes. Elevated glucose in CHD group indicates risk of co-morbid conditions.

#### Chart - 10

In [ ]:
# Chart - 10 visualization code
sns.boxplot(x='TenYearCHD',y='heartRate',data=df,palette='cool')
plt.title('Heart Rate vs CHD')
plt.xticks([0,1],['No CHD','CHD'])
plt.ylabel('Heart Rate')
plt.show()

##### 1. Why did you pick the specific chart?

Heart rate could indicate stress on the heart.

##### 2. What is/are the insight(s) found from the chart?

CHD patients tend to have a slightly higher median heart rate.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Yes. Adds value in monitoring early stress or cardiac issues,
No. Only slight difference; not a strong negative growth factor.

#### Chart - 11

In [ ]:
# Chart - 11 visualization code
sns.countplot(x='education',hue='is_smoking',data=df,palette='Set3')
plt.title('Education vs Smoking Habits')
plt.xlabel('Education Level')
plt.ylabel('Count')
plt.show()

##### 1. Why did you pick the specific chart?

Education may relate to lifestyle habits.

##### 2. What is/are the insight(s) found from the chart?

Lower education levels are associated with higher smoking rates.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Yes. Enables targeted education-based smoking cessation programs,
Yes. Lower education levels link to higher smoking, worsening long-term health.

#### Chart - 12

In [ ]:
# Function for Plotting the continuous variable distribution along with their median and mean
def displot_with_median(dataset, variable, median = False, mean = False, unit = None):
  '''A displot with median and mean and the appropriate units (if any) as the inputs'''
  sns.displot(dataset[variable], height = 5, aspect = 11/6, bins = 40, kde = True)
  if median == True:
    plt.axvline(dataset[variable].median(), color = 'red', linestyle = '--', label = f'Median = {dataset[variable].median():.2f} {unit}')
  if mean == True:
    plt.axvline(dataset[variable].mean(), color = 'green', linestyle = '--', label = f'Mean = {dataset[variable].mean():.2f} {unit}')
  plt.ylabel('Count of people')
  plt.xlabel(var + f' ({unit})')
  plt.legend()
  plt.show()

# Units for each of the continuous variables
units = ['years', 'count/day', 'mg/dL', 'mmHg', 'mmHg', 'kg/m2', 'beats/min', 'mg/dL']
cont_var_units = dict(zip(cont_vars, units))

# Plotting the continuous variable distributions
for var in cont_var_units:
  displot_with_median(df, var, median = True, mean = True, unit = cont_var_units[var])

##### 1. Why did you pick the specific chart?

The distplot is useful for continuous variables because it provides a visual representation of the distribution of values in the variable across the dataset. This allows one to understand how the data is distributed, such as whether it follows a normal distribution, is skewed to the left or right, or has multiple peaks. It can also help to identify outliers and anomalies in the data, which can be further investigated and addressed as needed.

##### 2. What is/are the insight(s) found from the chart?

* From the plots, most continuous variables appear to be right-skewed (tail towards higher values), as indicated by the median being less than the mean. Some variables show outliers that should be carefully checked to confirm if they are valid.

* According to ACC/AHA guidelines, the optimal systolic and diastolic blood pressure levels are around 130 mmHg and 80 mmHg respectively. The median and mean blood pressure values in the dataset are close to or exceed these levels, suggesting that a majority of individuals may have elevated blood pressure.



#### Chart - 13

In [ ]:
# Defining a function to annotate the data values in the graph
def display_vals(axis, round_ = 2):
  '''Displays the data value on the chart'''
  for p in axis.patches:
   axis.annotate(str(round(p.get_height(), round_)), (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', xytext=(0, 10), textcoords='offset points')

In [ ]:
for var in categ_vars:
  plt.figure(figsize = (11, 6))
  ax = df[var].value_counts().plot(kind = 'bar')
  plt.ylabel('Count of people')
  plt.xlabel(var)
  display_vals(ax)

##### 1. Why did you pick the specific chart?

Barplots are useful for discrete variables because they allow us to visualize the frequency or proportion of each category in the variable occuring in the dataset, to identify any imbalances or patterns in the data, and for comparing these across different groups or subgroups in the dataset.

##### 2. What is/are the insight(s) found from the chart?

The following can be observed from the above plots:
*   There are **more number of females** present in the dataset than males.
*   Assuming that the values in Education feature are hierarchical in ascending order, **more number of people are less educated** in the dataset
*   **Only 100 people (~3% of dataset) are taking medications for Blood pressure** despite **over a 1000 people having history of Hypertension** and also, half of the people having systolic and diastolic Blood Pressure over the optimum 130mmHg/80mmHg respectively as seen in previous Chart
*   **Only 22 people have had a recorded history of stroke** (0.6% of dataset)
*   **Only 87 people have diabetes** (~2% of the dataset)

##### 3. Will the gained insights help creating a positive business impact?

The disproportion between the number of people taking medications for Blood Pressure and those with BP levels higher than optimum could be addressed on analysing these data. Further tests if necessary could be conducted on those with high BP and not taking medications, and to prescribe them any medications based on the results, if necessary.

#### Chart - 14 - Correlation Heatmap

In [ ]:
# Correlation Heatmap visualization code
corr=df.select_dtypes(include='number').corr()
sns.heatmap(corr,annot=True,cmap='RdBu_r',fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

##### 1. Why did you pick the specific chart?

Understand relationships between all numerical variables.

##### 2. What is/are the insight(s) found from the chart?

Age and systolic BP show notable correlation with CHD

#### Chart - 15 - Pair Plot

In [ ]:
# Pair Plot visualization code
sns.pairplot(df[['age','sysBP','totChol','glucose','TenYearCHD']],hue='TenYearCHD',palette='husl')
plt.suptitle('Pairplot of Key Features',y=1.02)
plt.show()

##### 1. Why did you pick the specific chart?

Visually compare relationships.

##### 2. What is/are the insight(s) found from the chart?

Some separation between CHD and non-CHD groups is visible across variables.

## ***5. Hypothesis Testing***

### Based on your chart experiments, define three hypothetical statements from the dataset. In the next three questions, perform hypothesis testing to obtain final conclusion about the statements through your code and statistical testing.

### Hypothetical Statement - 1

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

According to [Centers for DIsease Control and Prevention](https://www.cdc.gov/bloodpressure/facts.htm#:~:text=Nearly%20half%20of%20adults%20in,are%20taking%20medication%20for%20hypertension.), 44% of women have high blood pressure - defined as a Systolic BP higher than 130mmHg or Diastolic BP higher than 90mmHg - while 50% of men have the same. These 2 shall form the basis of the two Hypothesis tests in this section.

The First Hypothesis Test is for:
*   Null Hypothesis 1: 44% of women have high BP
*   Alternate Hypothesis: Proportion of women having high BP is not equal to 44%

The Second Hypothesis Test is for:
*   Null Hypothesis 2: 50% of men have high BP
*   Alternate Hypothesis: Proportion of men having high BP is not equal to 50%

For this, the one-sample proportion test is used. The test is double tailed with significance value of 0.05

#### 2. Perform an appropriate statistical test.

In [ ]:
# Performing the one-sample proportion tests
h_vals = {'F':0.44, 'M':0.50}
c = 1
for sex in h_vals:
  df_h1 = df[df['sex'] == sex]
  n_sex = df_h1.shape[0]
  n_sex_highbp = df_h1[(df_h1['sysBP'] >= 130) | (df_h1['diaBP'] >= 90)].shape[0]
  alpha = 0.05
  stat, p_value = proportions_ztest(count = n_sex_highbp, nobs = n_sex, value = h_vals[sex])
  x = 'Women' if sex == 'F' else 'Men'
  print(f'Hypothesis test {c}:')
  if p_value < alpha:
    print(f'P-value = {p_value} < {alpha}. The Null Hypothesis stands rejected. The true proportion of {x} having high Blood Pressure is greater than {h_vals[sex] * 100}%\n')
  else:
    print(f'P-value = {p_value} > {alpha}. The Null Hypothesis that the true proportion of {x} with high Blood pressure is {h_vals[sex] * 100}% is failed to be rejected')
  c+= 1

It is to be noted that the Framingham test data is old, and older than the data provided in the link directing to Center for Disease Control, which is updated till the previous year. Hence, one of the tests (specifically, for the female) did not match up to the Null Hypothesis statement

##### Which statistical test have you done to obtain P-Value?

The **one-sample proportion test** is utilised for this statistical Hypothesis test. One-sample proportion test is used to determine if a proportion of a single sample differs significantly from a hypothesized value. It is also known as the one-sample z-test for proportions. This test is typically used to test a hypothesis about a population proportion based on a sample proportion.

##### Why did you choose the specific statistical test?

The test assumes that the sample is randomly selected and that the sample size is large enough to meet the conditions for a normal approximation of the sampling distribution, i.e., it is not necessary that the sample or population be normally distributed. The one-sample proportion test is used to test hypotheses about a single proportion or percentage, and it is based on the binomial distribution. Since the hypothesized values were for testing proportions, this particular test was utilised

### Hypothetical Statement - 2

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

This Hypothesis test will be based on the assumption that the median population will have the optimum values for Blood Pressure - i.e., 130/80 mmHg for the Systolic Blood Pressure and Diastolic Blood Pressure respectively. Thus, the Null Hypotheses will be on a metric which is a consolidation of these two metrics - the Mean Arterial Pressure (a weighted average of sys BP and dia BP).

Thus, the optimal MAP taken for this test is (2*80 + 130)/3 = 96.667

*   Null Hypothesis: Mean MAP of the population is 96.667mmHg
*   Alternative Hypothesis: Mean MAP of the population is not equal to 96.667mmHg

Test summary:
*   The one-sample t-test will be used
*   The test is two-tailed with a significance value of 0.05
*   Since the t-test is used, it is required that the data is normally distributed

#### 2. Perform an appropriate statistical test.

In [ ]:
df_h2 = df[['sysBP', 'diaBP']]
df_h2['MAP'] = (df['sysBP'] + df['diaBP'] * 2)/3
h2_val = 96.667
alpha2 = 0.05

The Shapiro test shall be used to check the normal distribution of the MAP feature with a significance value of 0.05, since normal distribution is a requirement for this test

In [ ]:
print(f"P-value for checking Normal distribution using shapiro test = {shapiro(df_h2['MAP'])[1]}")

In [ ]:
var = 'MAP'
displot_with_median(df_h2, var, median = True, mean = True, unit = 'mmHg')

Since this p-value is very low, we fail to reject the Null Hypothesis that the data is normally distributed, and the data has to be transformed to make it normally distributed. The inverse transform is used since the data has a left skew

In [ ]:
df_h2['MAP'] = 1/df_h2['MAP']

In [ ]:
print(f"p value for checking Normal distribution using shapiro test = {shapiro(df_h2['MAP'])[1]}")

In [ ]:
var = 'MAP'
displot_with_median(df_h2, var, median = True, mean = True, unit = '1/(mmHg)')

Now that the data is normally distributed, with a p-value from the shapiro test greater than a required significance value, the requirement for the t-test is fulfilled. The Null Hypothesis value of MAP also has to be applied this inverse transformation to maintain consistency. The t-test is applied in the below code

In [ ]:
p_value_h2 = ttest_1samp(a = df_h2['MAP'], popmean = 1/h2_val)[1]
if p_value_h2 < alpha2:
  print(f'P-value = {p_value_h2} < {alpha2}. The Null Hypothesis stands rejected. The true mean of Mean Arterial Pressure is not equal to the optimum {h2_val}mmHg')
else:
  print(f'P-value = {p_value_h2} > {alpha2}. The Null Hypothesis that the true mean of Mean Arterial Pressure is equal to {h2_val}mmHg is failed to be rejected')

##### Which statistical test have you done to obtain P-Value?

Two statistical tests were applied in this Hypothesis test:
*   **Shapiro test** to check whether the MAP values are normally distributed
*   **One-sample t-test** to check whether the true mean of the MAP was indeed equal to the Hypothesised value

##### Why did you choose the specific statistical test?

The one-sample t-test is used here for the following reasons:
*  The goal was to check the hypothesis statement about the true mean of a metric
*  The Population variance of the same metric (MAP) was not known

In such a case, the t-test is usually relied on.

Also, it is necessary that the data be normally distributed in order to apply a t-test on it. Hence, the Shapiro test was utilised to check whether the data is normally distributed, and if not, to apply the relevant transformations to make it normally distributed for the one-sample t-test

### Hypothetical Statement - 3

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Null Hypothesis (H₀): The average age of patients with CHD is equal to those without CHD.

Alternative Hypothesis (H₁): The average age of patients with CHD is significantly higher than those without CHD.


#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value
from scipy.stats import ttest_ind
# Split age data into CHD and non-CHD groups
age_chd = df[df['TenYearCHD'] == 1]['age']
age_no_chd = df[df['TenYearCHD'] == 0]['age']
# Perform two-sample t-test
t_stat, p_value = ttest_ind(age_chd, age_no_chd, nan_policy='omit')
print("P-Value:", p_value)

##### Which statistical test have you done to obtain P-Value?

Independent Two-Sample t-test.

##### Why did you choose the specific statistical test?

Because we are comparing the average age between two independent groups (CHD vs non-CHD).

## ***6. Feature Engineering & Data Pre-processing***

In [ ]:
# Creating a copy of the data
data = df.copy()

### 1. Handling Missing Values

In [ ]:
print(f'There is a total of {df.isna().sum().sum()} missing values in the dataset. They are distributed among the variables as follows\n')

# Defining a function to annotate the data values in the graph
def display_vals(axis, round_ = 2):
  '''Displays the data value on the chart'''
  for p in axis.patches:
   axis.annotate(str(round(p.get_height(), round_)), (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', xytext=(0, 10), textcoords='offset points')

# Visualising the number of missing values in each variable
plt.figure(figsize = (11, 6))
ax = data.isna().sum().sort_values(ascending = False).plot(kind = 'bar')
plt.ylabel('Number of missing values')
display_vals(ax)

#### 1.1 Categorical variables

In [ ]:
plt.figure(figsize = (11, 6))
ax = data[categ_vars].isna().sum().sort_values(ascending = False).plot(kind = 'bar')
plt.ylabel('Number of missing values')
display_vals(ax)

Among the categorical variables, the **education** variable has been concluded to be dropped from the dataset. So, only **BPMeds** among the categorical variables contain missing values.

In [ ]:
categ_vars.remove('education')

In [ ]:
# Collecting the indices of rows where BPMeds is missing
bpmeds_missing = data[data['BPMeds'].isna()].index

[According to this article](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2560868/#:~:text=For%20most%20people%2C%20blood%20pressure%20readings%20should%20be,80%20mmHg%20when%20measured%20in%20the%20doctor%E2%80%99s%20office.) from the National Library of Medicine, BP medications are recommended by doctors to the following patients:
*   Having systolic and diastolic Blood Pressure levels above 140/90 mmHg
*   For those having diabetes or kidney diseases, having Blood pressure levels above 130/80 mmHg

Hence, the missing values in BPMeds, which are not large in number (1.2% of dataset), are filled with the above rules

In [ ]:
# Filling the BPMeds missing values as according to above
for indx in bpmeds_missing:
  # For diabetic patients
  if data.loc[indx, 'diabetes'] == 1 and (data.loc[indx, 'sysBP'] > 130 or data.loc[indx, 'diaBP'] > 80):
    data.loc[indx, 'BPMeds'] = 1

  # For non-diabetic patients
  elif data.loc[indx, 'diabetes'] == 0 and (data.loc[indx, 'sysBP'] > 140 or data.loc[indx, 'diaBP'] > 90):
    data.loc[indx, 'BPMeds'] = 1

  # For rest of the patients
  else:
    data.loc[indx, 'BPMeds'] = 0

In [ ]:
data[categ_vars].isna().sum()

All missing values from categorical variables have been removed

#### 1.2 Continuous variables

In [ ]:
plt.figure(figsize = (11, 6))
ax = data[cont_vars].isna().sum().sort_values(ascending = False).plot(kind = 'bar')
plt.ylabel('Number of missing values')
display_vals(ax)

For continuous variabes, **cigsPerDay** will be dealt with separately before handling the other variables

##### 1.2.1 cigsPerDay

Let us take a brief look at the relation between **cigsPerDay** and **is_smoking** (current smoker or not)

In [ ]:
# Checking unique values for cigsPerDay when is_smoking = "NO"
data[data['is_smoking'] == 'NO']['cigsPerDay'].value_counts()

In [ ]:
# Checking unique values for cigsPerDay when is_smoking = "YES"
data[data['is_smoking'] == 'YES']['cigsPerDay'].value_counts().sort_index().head(3)

In [ ]:
# Checking values for is_smoking wherever cigsPerDay has a missing value
data[data['cigsPerDay'].isna()]['is_smoking'].value_counts()

In [ ]:
# Checking the distribution
var = 'cigsPerDay'
displot_with_median(data, var, median = True, mean = True, unit = cont_var_units[var])

*   As can be observed, wherever **is_smoking = 'NO'**, the value of **cigsPerDay** is always **0**. **cigsPerDay** is never zero when **is_smoking = 'YES'**. Hence, a one-to-one mapped relationship exists between **is_smoking = 'NO'** and **cigsPerDay = 0**.
*   All the people where **cigsPerDay** has a missing value are current smokers (i.e., **is_smoking == 'YES'**)
*   Also, the distribution of **cigsPerDay** is extremely left skewed with a median of 0, indicating highest amount of non-smokers in the dataset

From the above observations, it would not make sense to impute the missing values for **cigsPerDay** with the median of the entire dataset. **Instead, median/mean of only the current smokers is more appropriate**

In [ ]:
# Checking distribution and outliers for only smokers
var = 'cigsPerDay'
fig, axx = plt.subplots(nrows = 1, ncols = 2, figsize = (16, 6))
sns.histplot(data[data['is_smoking'] == 'YES'][var], bins = 40, kde = True, ax = axx[0])
sns.boxplot(data[data['is_smoking'] == 'YES'], x = var, orient = 'h', ax = axx[1])
axx[1].set(ylabel = var, xlabel = cont_var_units[var])

**Since cigsPerDay contains outliers even among the smokers in the dataset, the median is chosen for imputation**

In [ ]:
# Imputing with median of current smokers
imputing_median1 = data[data['is_smoking'] == 'YES']['cigsPerDay'].median()
data['cigsPerDay'] = data['cigsPerDay'].fillna(imputing_median1)

In [ ]:
# Checking if any missing values are still present
data['cigsPerDay'].isna().sum()

##### 1.2.2 glucose, totChol, BMI and heartRate

In [ ]:
# Checking distribution and outliers
for var in ['glucose', 'totChol', 'BMI', 'heartRate']:
  fig, axx = plt.subplots(nrows = 1, ncols = 2, figsize = (16, 6))
  sns.histplot(data[var], bins = 40, kde = True, ax = axx[0])
  axx[0].axvline(df[var].median(), color = 'red', linestyle = '--', label = f'Median = {round(df[var].median(), 2)}')
  axx[0].axvline(df[var].mean(), color = 'green', linestyle = '--', label = f'Mean = {round(df[var].mean(), 2)}')
  axx[0].set(xlabel = var + f' ({cont_var_units[var]})')
  axx[0].legend()
  sns.boxplot(data, x = var, orient = 'h', ax = axx[1])
  axx[1].set(ylabel = var, xlabel = cont_var_units[var])

As observed from above,
*   The mean of each variable is slightly higher than the median indicating a right skew of varying degrees.
*   Each variable contains outliers even outside of twice the interquartile range.

For these reasons, missing values in each variable is imputed with the median of the respective variable



In [ ]:
# Imputing the missing values with the median for each variable
for var in ['glucose', 'totChol', 'BMI', 'heartRate']:
  imputing_median = data[var].median()
  data[var]  = data[var].fillna(imputing_median)

In [ ]:
# Checking for any missing values in the dataset
data.isna().sum()

All missing values have been removed except education which is going to be dropped

##### 1.2.3 Checking mean, median and distribution of continuous variables after imputing missing values

In [ ]:
for var in ['cigsPerDay', 'glucose', 'totChol', 'BMI', 'heartRate']:
  fig, axx = plt.subplots(nrows = 1, ncols = 2, figsize = (16, 6))
  sns.histplot(df[var], bins = 40, kde = True, ax = axx[0])
  sns.histplot(data[var], bins = 40, kde = True, ax = axx[1])
  for i, datas in enumerate([df, data]):
    axx[i].axvline(datas[var].median(), color = 'red', linestyle = '--', label = f'Median = {datas[var].median():.2f} {cont_var_units[var]}')
    axx[i].axvline(datas[var].mean(), color = 'green', linestyle = '--', label = f'Mean = {datas[var].mean():.2f} {cont_var_units[var]}')
    axx[i].set(xlabel = var + f' ({cont_var_units[var]})', ylabel = 'Count of people')
    axx[i].legend()
  axx[0].set(title = f'Input distribution of {var}')
  axx[1].set(title = f'Distribution of {var} after imputing null values')
  plt.legend()
  plt.show()

It can be observed that the mean, median and the overall distribution for each of the continuous variables are the same before and after imputation of missing values.

### 2. Feature Manipulation and Selection

Before analysing outliers, certain features need to be handled to reduce dimensionality . In this section, the redundant variables are removed and features with high correlation are dealt with.

#### 2.1 education, is_smoking and prevalentStroke

Right off the start, **education** and **is_smoking** can be removed from the dataset.

*   **Education** is removed because of the less information provided of the variable, and the low correlation with **TenYearCHD**.
*   **is_smoking** is removed because of the redundancy of information with respect to **cigsPerDay**. All the information contained within the former is already found in the latter, i.e., whenever **is_smoking = NO**, **cigsPerDay** has a value of zero, and whenever **is_smoking = YES**, **cigsPerDay** has a non-zero value .

In [ ]:
data = data.drop(['education', 'is_smoking'], axis = 1)

Also, **prevalentStroke** is removed because of the high class imbalance - only 0.5% of the dataset having one class, as shown below

In [ ]:
var = 'prevalentStroke'
plt.figure(figsize = (11, 6))
ax = data[var].value_counts().plot(kind = 'bar')
plt.ylabel('Count of people')
plt.xlabel(var)
display_vals(ax)

In [ ]:
data.drop('prevalentStroke', axis = 1, inplace = True)

#### 2.2 Systolic BP and Diastolic BP

*   Now, to address the high multicollinearity between systolic Blood Pressure and Diastolic Blood pressure, the two are combined to form another continuous variable named '**Mean Arterial Pressure**'
*   The Mean Arterial Pressure is a weighted average of diastolic and systolic Blood Pressures representing the average arterial pressure in one cardiac cycle. The formula is given as

\begin{align}
MAP = \frac{2*DiastolicBP + Systolic BP} 3
\end{align}

*   The MAP is a better predictor of Cardiovascular disease than Pulse Pressure, which is the other variable combining the systolic and diastolic Blood Pressures. Hence, the MAP is chosen to resolve the multicollinearity between **sysBP and diaBP**

In [ ]:
data['MAP'] = (data['sysBP'] + 2*data['diaBP'])/3

In [ ]:
# Checking the distribution
displot_with_median(dataset = data, variable = 'MAP', median = True, mean = True, unit = 'mmHg')

#### 2.3 prevalentHyp

In [ ]:
# Checking distribution of prevalentHyp

filter_counts = data[data['MAP'] > 100]['prevalentHyp'].value_counts().sort_index()
all_counts = data['prevalentHyp'].value_counts().sort_index()
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (14, 6))
ax[0].bar(all_counts.index, all_counts.values)
ax[0].set_title('All Data')
ax[1].bar(filter_counts.index, filter_counts.values)
ax[1].set_title('MAP > 100mmHg')
for i in range(2):
  ax[i].set(xticks = [0, 1], ylim = [0, 2500], ylabel = 'Count of people', xlabel = 'prevalentHyp')
  display_vals(ax[i])
plt.tight_layout()
plt.show()

In [ ]:
from scipy.stats import pointbiserialr
corr, p_value = pointbiserialr(data['prevalentHyp'], data['MAP'])
print(f"Point biserial correlation: {corr}, p-value: {p_value}")


print()
correlation = data[['prevalentHyp', 'MAP']].corr()
print(correlation)



* A correlation of ~0.69 or higher suggests moderate to strong linear relationship.


*   correlation is high and statistically significant (p-value < 0.05)
*   **prevalentHyp** is deemed redundant and dropped from the analysis

#### 2.4 Glucose and Diabetes

*   **Glucose and diabetes** are directly linked to each other in that, high levels of glucose indicate high likelihood of diabetes. While there are other factors which cause elevated glucose levels even for non-diabetic patients, like stress, certain medications, and some medical conditions, it is not common.
*    An analysis between the relationship of glucose and diabetes in this dataset is conducted based on the above factors.

In [ ]:
from scipy.stats import pointbiserialr

corr, p_value = pointbiserialr(data['diabetes'], data['glucose'])
print(f"Point Biserial Correlation: {corr:.3f}, p-value: {p_value:.3e}")


The diabetes column can be dropped as it is redundant, given its high correlation with glucose. This suggests that glucose alone provides similar predictive information.

The glucose variable contains a large number of outliers, which will be handled by transforming it into a categorical variable called diabetes_grade, enabling more robust analysis and modeling.

In [ ]:
# Defining the function for Diabetes grades
def diabetes_grades(df):
  if df['glucose'] >= 126:
    return 4 #Diabetes
  elif df['glucose'] > 100:
    return 3 #Pre-diabetes
  elif df['glucose'] > 70:
    return 2 #Normal
  elif df['glucose'] < 71:
    return 1 #Hypoglycemia

In [ ]:
data['diabetes_grade'] = data.apply(diabetes_grades, axis = 1)

#### 2.6 Final features

In [ ]:
# Dropping the redundant columns
data = data.drop(['sysBP', 'diaBP', 'glucose', 'diabetes', 'prevalentHyp'], axis = 1)
cont_vars = [var for var in cont_vars if var not in ['sysBP', 'diaBP', 'glucose']]
cont_vars+=['MAP']
categ_vars = [var for var in categ_vars if var not in ['is_smoking', 'diabetes', 'prevalentStroke', 'prevalentHyp']]
categ_vars+=['diabetes_grade']

print(f'The categorical variables are: {categ_vars}')
print(f'The continuous variables are: {cont_vars}')

In [ ]:
# Checking the correlation between variables
corr=data.select_dtypes(include='number').corr()
sns.heatmap(corr,annot=True,cmap='RdBu_r',fmt=".2f")
plt.title('Correlation Matrix')
plt.show()


As can be seen above, all multicollinearity has been removed .

In summary, this section consisted of:
*   dropping **education**, **is_smoking** and **prevalentStroke**
*   combining **sysBP** and **diaBP** to get **MAP**
*   modifying **glucose** to get **diabetes_grade**
*   dropping **diabetes** and **prevaletHyp**

### 3. Handling Outliers

*   It is important to note that while outliers may impact the accuracy of the model, they have to be dealt with carefully as some outliers may be realistic and within the possible range of values of the particular feature, and not due to any measurement error.
*   At the same time, extreme outliers even if they are possible and true values, do need to be dealt with as it impacts the predictive power of the Machine Learning model which is sensitive to outliers.
*   Since many continuous variables have a positive correlation with the dependent variable **TenYearCHD**, a bivariate outlier analysis has to be done.

In [ ]:
# Visualising the outliers
cont_var_units['MAP'] = 'mmHg'
for var in cont_vars:
  plt.figure(figsize = (13,7))
  sns.boxplot(data = data, y = var, x = 'TenYearCHD')
  plt.ylabel(var)
  plt.xlabel(cont_var_units[var])
  plt.show()

*   It can be observed that
  *   **age** has no outliers
  *   **cigsPerDay** has outliers of about **60 and 70 cigs/day**
  *   **totChol** has outliers with maximum of around **700mg/dL**, and minimum around **100mg/dL**
  *   There are patients with **BMI** even above **50**
  *   **heartRate** has outliers with maximum of around **140 beats/day**, and with minimum below **50 beats/day**
  *   **MAP** also has outliers with maximum values close to **190mmHg**
  
*   All these figures are within the possible range of values for each respective feature. But even if they're realistic, they are extremely rare and this may affect the model prediction power
*   Since they are within the possible range of values, the outliers shall not be trimmed since that may lead to loss of data and make the ML models biased. Instead, they shall be limited to a particular value, which though are less likely to occur are not as rare as the maximum outliers. This process is called **Winsorising**.
  *   **cigsPerDay** shall be limited to **50 cigs/day**
  *   **totChol - 500mg/dL**
  *   **BMI - 45 (>45 is extremely obese)**
  *   **heartRate** in the range of **50-120 beats/day**
  *   **MAP - 165mmHg**, beyond which it is a medical emergency
  *   **NOTE**: **Cholestrol level, MAP and heartRate** above **500mg/dL, 120beats/day and 165mmHg** respectively are considered medical emergencies and require immediate medical attention


In [ ]:
# Limitting maximum values
max_limits = [50, 500, 45, 120, 165]
outlier_vars = cont_vars[1:]
for var, limit in zip(outlier_vars, max_limits):
  data.loc[data[var] > limit, var] = limit

# Limitting the minimum value of heartRate
data.loc[data['heartRate'] < 50, 'heartRate'] = 50

In [ ]:
# Visualising the outliers with the maximum limit after Winsorising
for var, limit in zip(outlier_vars, max_limits):
  plt.figure(figsize = (13,7))
  sns.boxplot(data = data, y = var, x = 'TenYearCHD')
  plt.axhline(limit, color = 'red', linestyle = '--')
  plt.ylabel(var)
  plt.xlabel(cont_var_units[var])
  plt.show()

All the datapoints for each variable have now been contained within the chosen limits

##### What all outlier treatment techniques have you used and why did you use those techniques?

As stated earlier, **Winsorising** was used to handle the outliers in this dataset. Winsorizing is preferred over Trimming when the number of outliers is small, and the range of values is not unrealistically extreme. Winsorising replaces the extreme values with a value within the range of the data, which preserves the original distribution of the data. In contrast, trimming completely removes the extreme values from the data, which can alter the distribution of the data. Since the datapoints here were not outside of the possible range of each feature, and were not large in number, Winsorising was used.



### 4. Categorical Encoding

In [ ]:
data.info()

In [ ]:
data['sex'].value_counts()

As seen from above, **sex** is still of object type. Since there are only 2 classes, this feature is encoded as per the following rule
*   **Male - 1**
*   **Female - 0**

In [ ]:
# Encoding sex column
data['sex'] = data['sex'].map({'M':1, 'F':0})

In [ ]:
data.info()

All variables are now in numerical datatype

#### What all categorical encoding techniques have you used & why did you use those techniques?

One-Hot Encoding was used here since the sex variable had only 2 classes. Essentially, what this feature represents is whether a patient is male or not by assigning a non-zero value (one) if it is male.

### 5. Data Splitting

In this case, data is not transformed because outliers were handled without the need to change the data distributions. So, the splitting of the data is directly performed

In [ ]:
X = data.drop(['TenYearCHD', 'id'], axis = 1)
Y = data['TenYearCHD']

# Visualising the input data
X.head()

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 8, stratify = Y, shuffle = True)

In [ ]:
Y_train.value_counts()

In [ ]:
Y_test.value_counts()

##### What data splitting ratio have you used and why?

Because there is not a large number of data, and with 9 independent input features, a larger train-test split of 80-20 is chosen to allow the model to learn from a more diverse set of examples. Since it is an unbalanced problem, the split is stratified with respect to the dependent variable **TenYearCHD**

### 6. Handling Imbalanced Dataset

##### Do you think the dataset is imbalanced? Explain Why.

Yes, the dataset is imbalanced since the classes in the variable to be predicted are not equally distributed. There are more number of people who do not have a risk of CHD compared to those who do. This can be seen in the below plot of the train data

In [ ]:
plt.figure(figsize = (11, 6))
ax = Y_train.value_counts().plot(kind = 'bar')
plt.ylabel('Count of people')
plt.xlabel('TenYearCHD')
display_vals(ax)

In [ ]:
# Handling Imbalanced Dataset
smote = SMOTE(random_state = 8)
X_smote, Y_train_final = smote.fit_resample(X_train, Y_train)

In [ ]:
# Visualising the class balance after using SMOTE
plt.figure(figsize = (11, 6))
ax = Y_train_final.value_counts().plot(kind = 'bar')
plt.ylabel('Count of people')
plt.xlabel('TenYearCHD')
display_vals(ax)

##### What technique did you use to handle the imbalance dataset and why?

Synthetic Minority Over-sampling Technique (SMOTE) is a popular oversampling method for handling class imbalance. It is used for the following reasons
*   Since not much data is present for training, undersampling will not be a good method as it may lead to loss of data and hence leading to underfitting of the model
*   Also, because there is a large imbalance (82-18% imbalance) simply replicating minority class values to create a balanced dataset will lead to overfitting of the model
*   Instead, SMOTE is used which synthetically creates new datapoints using the k-nearest neighbours, hence creating new realistic data to train the models on.

### 7. Data Scaling

In [ ]:
# Scaling the train and test data according to train data
scaler = MinMaxScaler()
X_train_final = scaler.fit_transform(X_smote)
X_test_final = scaler.transform(X_test)

##### Which method have you used to scale you data and why?

*   The Minmaxscaler is usually used when the lower and upper boundaries in the area of interest are well-known
*   It also helps to maintain the the interpretability of the values after scaling
*   MinMaxScaler also helps maintain the relative relationship between the datapoints, which is crucial in this case of CHD risk prediction
*   Also since there is no specific requirement of the mean and standard deviation of the data, the MinMaxScaler is used

## ***7. ML Model Implementation***

The following 3 models are implemented in this section on the train dataset, and then tested on the test data.
*   Logistic Regression
*   Decision Tree
*   XGBoost

The scoring metric preferred for evaluation is the **Recall**. The **ROC-AUC score** is also recorded for each model.


### Functions to train each ML Model

In [ ]:
# Function to plot the Confusion Matrix
def confusion_plot(cm):
  '''Plots the Confusion Matrix given as input'''
  cmd = ConfusionMatrixDisplay(cm, display_labels = ['No risk (0)', 'Risk (1)'])
  cmd.plot(cmap = 'Blues')
  plt.title('Confusion Matrix for Test Data')
  plt.show()

# Function to train and test a given classification model
def model_train_test(model, train_x, train_y, test_x, test_y, gs = False, confusion = True):
  '''Trains the classification model given as input. Other inputs include Test and train data
  and a Boolean to inform the function if GridSearch is being performed
  Returns the train and test Recalls and ROC-AUC scores, the test data predictions, and the final model'''

  model.fit(train_x, train_y)
  if gs == True:
    print(f'Best model parameters are: {model.best_params_}')
    print(f'Best model score is: {model.best_score_}\n')
    model = model.best_estimator_

  # Getting the train and test predictions
  train_preds = model.predict(train_x)
  train_recall = recall_score(y_true = train_y, y_pred = train_preds, average='binary')
  train_roc = roc_auc_score(train_y, train_preds)
  test_preds = model.predict(test_x)
  test_recall = recall_score(y_true = test_y, y_pred = test_preds, average='binary')
  test_roc = roc_auc_score(test_y, test_preds)

  # Plotting confusion matrix
  if confusion == True:
    confusion_plot(confusion_matrix(test_y, test_preds))

  output_metrics = {'Train Recall':train_recall, 'Test Recall':test_recall, 'Train ROC-AUC':train_roc, 'Test ROC-AUC':test_roc}
  return output_metrics, test_preds, model

In [ ]:
# Creating a dictionary of lists to store Train and test Recalls and ROC-AUC scores
scores = dict()
scores['Train Recall'] = []
scores['Test Recall'] = []
scores['Train ROC-AUC'] = []
scores['Test ROC-AUC'] = []
model_names = ['Logistic Regression', 'XGBoost' ,'Decision Tree']

### ML Model - 1 - Logistic Regression

In [ ]:
# Training the model
lr_scores, lr_test_preds, lr_model = model_train_test(LogisticRegression(), X_train_final, Y_train_final, X_test_final, Y_test)

In [ ]:
# Classification Report
print(classification_report(Y_test, lr_test_preds, target_names=['class-0', 'class-1']))

In [ ]:
# Printing the train and test Recalls and ROC-AUC scores
def print_scores(model_name, model_scores):
  '''Function to print the scores of a given model'''
  print(f"The train and test recalls of the {model_name} Model are: {round(model_scores['Train Recall'] * 100, 2)}% and {round(model_scores['Test Recall'] * 100, 2)}% respectively")
  print(f"The train and test ROC-AUC scores of the {model_name} Model are: {round(model_scores['Train ROC-AUC'] * 100, 2)}% and {round(model_scores['Test ROC-AUC'] * 100, 2)}% respectively")

print_scores(model_name = model_names[0], model_scores = lr_scores)

In [ ]:
# Displaying feature importances
lr_importances = pd.Series(abs(lr_model.coef_[0]), index = X.columns)
plt.figure(figsize = (11, 6))
ax = lr_importances.sort_values(ascending = False).plot(kind = 'bar')
ax.set(xlabel = 'Features', ylabel = 'Feature importances')
display_vals(ax)

The following results can be observed from the Logistic Regression model:
*   **Train recall of 66.78% and Test Recall of 69.6%**
*   **Train ROC-AUC score of 65.85% and Test Recall of 68.83%**
  *   A test score higher than train score is not uncommon. Since the value is not significantly higher than the train recall, the reasons for this could be the test set having a different distribution of data than the training set, or the model performing better on the test set due to chance
*   **31 predictions out of all the 102 patients having risk of CHD were false**
*   **age is the most important feature for risk prediction of CHD, while BMI is the least important**

In [ ]:
for score in lr_scores:
  scores[score].append(lr_scores[score] * 100)

### ML Model - 2 - XGBoost

In [ ]:
# Defining the Hyperparameters and scoring metric
cv = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 3, random_state = 42)
scorer = make_scorer(recall_score, average = 'binary')



In [ ]:
# Defining the Hyperparameters
params_xgb = {
              'n_estimators':[50, 100],
              'max_depth':[3, 4],
              'learning_rate':[0.01, 0.02]
              }

xgb_model = xgb.XGBClassifier(random_state = 42)
xgb_model = GridSearchCV(xgb_model, params_xgb, cv = cv, scoring = scorer)

In [ ]:
# Training the model
%%time
xgb_scores, xgb_test_preds, xgb_model = model_train_test(xgb_model, X_train_final, Y_train_final, X_test_final, Y_test, gs = True)

In [ ]:
# Classification Report
print(classification_report(Y_test, xgb_test_preds, target_names=['class-0', 'class-1']))

In [ ]:
# Printing model scores
print_scores(model_name = model_names[2], model_scores = xgb_scores)

In [ ]:
# Displaying feature importances
xgb_importances = pd.Series(xgb_model.feature_importances_, index = X.columns)
plt.figure(figsize = (11, 6))
ax = xgb_importances.sort_values(ascending = False).plot(kind = 'bar')
ax.set(xlabel = 'Features', ylabel = 'Feature importances')
display_vals(ax)

The following results can be observed from the XGBoost model:
*   **Train recall of 80.55%  and Test Recall of  78.43%**
*   **Train ROC-AUC score of 67 % and Test Recall of 66.21%**
*   **22 predictions out of all the 102 patients having risk of CHD were false**
*   **The XGBoost model also predicts age to be the most important feature for risk prediction of CHD, while BMI, heartRate and sex are the least important**

In [ ]:
for score in xgb_scores:
  scores[score].append(xgb_scores[score] * 100)

### ML Model - 3 - Decision Tree

In [ ]:
# Defining the Hyperparameters
params_dt = {
              'max_depth' : [3, 4, 5],
              'min_samples_split':[10, 20, 25, 30],
              'min_samples_leaf':[10, 20, 25, 30]
              }

dt_model = DecisionTreeClassifier(criterion= 'entropy', random_state = 42)
dt_models = GridSearchCV(dt_model, params_dt, cv = cv, scoring = scorer)

In [ ]:
# Training the model
%%time
dt_scores, dt_test_preds, dt_model = model_train_test(dt_models, X_train_final, Y_train_final, X_test_final, Y_test, gs = True)

In [ ]:
# Classification Report
print(classification_report(Y_test, dt_test_preds, target_names=['class-0', 'class-1']))

In [ ]:
# Printing the model scores
print_scores(model_name = model_names[2], model_scores = dt_scores)

In [ ]:
# Displaying the Feature importances
dt_importances = pd.Series(dt_model.feature_importances_, index = X.columns)
plt.figure(figsize = (11, 6))
ax = dt_importances.sort_values(ascending = False).plot(kind = 'bar')
ax.set(xlabel = 'Features', ylabel = 'Feature importances')
display_vals(ax)

In [ ]:
# Visualising the decision tree
from sklearn import tree
graph = Source(tree.export_graphviz(dt_model, out_file = None, feature_names = X.columns, class_names=['0', '1'] , filled = True))
display(SVG(graph.pipe(format = 'svg')))



The following results can be observed from the Decision Tree model:
*   **Train recall of 90.19% and Test Recall of 87.25%**
*   **Train ROC-AUC score of 66.22% and Test Recall of 64.55%**
*   **13 predictions out of all the 102 patients having risk of CHD were false**
*   **age is again the most important feature for risk prediction of CHD, while sex, BMI and diabetes_grade are the least important. Since depth of the tree was 4 (as also seen in the plot above), the model did not split on the latter 3 variables**

In [ ]:
for score in dt_scores:
  scores[score].append(dt_scores[score] * 100)

### 1. Which HyperParameter Tuning and Cross Validation Technique have you used and why?

#### Hyperparameter Tuning Technique

*   Algorithms like **GridSearch** was introduced to automate the process of running the model on various desired combinations of components that the user wishes to optimise, instead of manually running through every combination which is not only time consuming, but also requires regular involvement of the user.

*   In this case, the **GridSearch** was used since the maximum time taken up by a model for training was 11mins (by Random Forest). While initially models like XGBoost took up a lot of time, it produced results which were overfitting. Hence, the *n_estimators, max_depth* parameters were brought down in number, thus considerably reducing the model training time. For these reasons, GridSearch was used.

#### Cross Validation Technique

If the hyperparameter tuning is performed on an imbalanced dataset, it is generally recommended to use **RepeatedStratifiedKFold** instead of StratifiedKFold. This is because the former provides a more robust estimate of model performance by repeating the cross-validation process with different random splits of the data. This can help to mitigate the impact of class imbalance on the performance estimate.

### 2. Which Evaluation metrics did you consider for a positive business impact and why?

*   As stated earlier, the **Recall** was chosen as the evaluation metric for comparison between models. The **Recall** is defined as below
\begin{align}
Recall = \frac{TruePositive} {True Positive + False Negative}
\end{align}
where **True Positive** is the number of correctly predicted patients with CHD risk, while **False Negative** is the number of patients who have a risk of CHD but were wrongly predicted to not have any.

*   Recall is a useful metric in CHD risk prediction because it focuses on the model's ability to correctly identify individuals who are at high risk for CHD. In other words, recall measures the proportion of individuals correctly identified as being at high risk out of all individuals who actually have high cardiovascular risk.

*   In the context of cardiovascular risk prediction, it is important to identify individuals who are at high risk so that appropriate interventions can be taken to prevent or manage their risk. False negatives (i.e., individuals who are at high risk but are not correctly identified as such) can lead to missed opportunities for prevention or treatment, and may result in adverse health outcomes. Therefore, a high recall rate is desirable in cardiovascular risk prediction models.


### 3. Which ML model did you choose from the above created models as your final prediction model and why?

In [ ]:
# Comparing the evaluation metrics from each model
results_df = pd.DataFrame(scores, index = model_names)
results_df

In [ ]:
# # Saving the result dataframe
results_df.to_csv('results_df.csv')

In [ ]:
# Plotting the train and test Recalls and choosing optimum model
results_df[['Test Recall', 'Train Recall']].plot(kind = 'barh', figsize = (11, 6))
plt.xlabel('Recall (%)')
plt.xlim((0, 100))
plt.xticks([0, 20, 40, 60, 80, 100, results_df['Test Recall'].max()])
plt.axvline(results_df['Test Recall'].max(), color = 'red', linestyle = '--')
plt.legend(bbox_to_anchor = (1, 0.5))
plt.show()

best_model = results_df[results_df['Test Recall'] == results_df['Test Recall'].max()].index[0]
print(f'\nThe model with maximum Test Recall is the {best_model} model')

The **Decision Tree** Model is the best model in this case when compared on the basis of **Test Recall**. It is to be noted that, while this model may not have the highest **Test ROC-AUC score**, almost all the models scored at a similar range in this metric (~65%). Hence, only comparing the **Test Recalls** would be apt in this case.



### 4. Explain the model which you have used and the feature importance using any model explainability tool?

*   Even though the model can be interpreted using the Decision tree plot earlier, using a model explainability tool can help intuitively visualise and interpret the model predictions
*   The explainability tool used here is **SHapley Additive exPlanations**, or **SHAP** in short. It is chosen because it is a very consistent, unbiased explainability tool which provides intuitive visualisation of model predictions which allow users to easily understand the contribution of each feature to the final prediction
*   **SHAP** provides insights into the overall behavior of the model.

In [ ]:

explainer = shap.TreeExplainer(dt_model)


shap_values = explainer.shap_values(X_test_final)

print(f"SHAP values type: {type(shap_values)}")
if isinstance(shap_values, list):
    print(f"SHAP values length: {len(shap_values)}")
    print(f"Class 0 shape: {shap_values[0].shape}")
    print(f"Class 1 shape: {shap_values[1].shape}")

if isinstance(shap_values, list) and len(shap_values) == 2:
    shap_values = shap_values[1]
elif len(shap_values.shape) == 3:
    shap_values = shap_values[..., 1]
print(f"Final SHAP values shape: {shap_values.shape}")
print(f"Number of features: {len(X.columns)}")

mean_shap = pd.Series(np.abs(shap_values).mean(axis=0))

shap_df = pd.DataFrame({
    'features': X.columns,
    'importance': mean_shap
}).sort_values('importance', ascending=False)


plt.figure(figsize=(10, 6))
plt.barh(shap_df['features'], shap_df['importance'], color='#1f77b4')
plt.xlabel('Mean Absolute SHAP Value (Impact on CHD Risk)')
plt.title('Feature Importance for CHD Risk Prediction')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

The feature importances plot shows that age is the most important factor for a higher risk of CHD, while sex, BMI and diabetes levels are the least important. This is a similar result as the Feature importances plot in the [Decision Tree model training section](#scrollTo=UNT3d48uGGyz&line=4&uniqifier=1). Both predict that age is the most important factor.

## ***8.*** ***Future Work (Optional)***

### 1. Save the best performing ml model in a pickle file or joblib file format for deployment process.


In [ ]:
# Save the File
import joblib
joblib.dump(dt_model, 'dt_model_final.pkl')

### 2. Again Load the saved model file and try to predict unseen data for a sanity check.


In [ ]:
# Load the File and predict unseen data.
model = joblib.load('dt_model_final.pkl')

### ***Congrats! Your model is successfully created and ready for deployment on a live server for a real user interaction !!!***

# **Conclusion**



The **Cardiovascular Risk Prediction** project aimed to develop a machine learning model to predict the 10-year risk of Coronary Heart Disease (CHD) using the Framingham dataset. The project involved comprehensive data preprocessing, feature engineering, and the implementation of multiple classification models to achieve the best predictive performance. Below are the key takeaways:

### **Key Insights and Findings:**
1. **Data Preprocessing:**
   - Missing values were imputed using realistic, research-backed methods (e.g., median imputation for continuous variables and logical rules for categorical variables like `BPMeds`).
   - Outliers were handled using **Winsorising** to retain data integrity while limiting extreme values.
   - Redundant features like `education`, `is_smoking`, and `prevalentStroke` were dropped to reduce dimensionality.
   - Highly correlated features (`sysBP` and `diaBP`) were combined into **Mean Arterial Pressure (MAP)** to mitigate multicollinearity.

2. **Feature Engineering:**
   - **Glucose** levels were transformed into a categorical variable (`diabetes_grade`) to better capture risk categories (e.g., normal, pre-diabetes, diabetes).
   - **Age** emerged as the most significant predictor of CHD risk, followed by `cigsPerDay` and `MAP`.

3. **Model Performance:**
   - Three models were evaluated: **Logistic Regression**, **Decision Tree**, and **XGBoost**.
   - **Decision Tree** outperformed others with the highest **Test Recall (87.25%)**, ensuring the model correctly identified the majority of high-risk patients.
   - **XGBoost** also performed well but was slightly less effective in recall compared to the Decision Tree.
   - **Logistic Regression** provided interpretable results but had lower recall, making it less suitable for this use case.

4. **Model Explainability:**
   - **SHAP (SHapley Additive exPlanations)** was used to interpret the Decision Tree model, confirming that **age**, **smoking habits**, and **blood pressure** were the most influential factors in predicting CHD risk.
   - The model’s transparency helps healthcare professionals understand risk factors and make informed decisions.

### **Business Impact:**
- The model can assist healthcare providers in **early identification of high-risk patients**, enabling timely interventions and preventive care.
- By focusing on **recall**, the model minimizes **false negatives**, ensuring fewer high-risk cases are missed.
- The insights can guide **personalized treatment plans** and **public health strategies** to reduce CHD incidence.

### **Future Work:**

- **Feature Expansion:** Incorporate more clinical and lifestyle factors (e.g., diet, exercise) for improved accuracy.
- **Deployment:** Integrate the model into healthcare systems for real-time risk assessment.

### **Final Takeaway:**
The **Decision Tree model** was selected as the best-performing model due to its high recall and interpretability. This project highlights the importance of **data preprocessing, feature engineering, and model selection** in building an effective predictive healthcare tool. The insights gained can significantly contribute to **reducing cardiovascular risks** through early detection and intervention.  

